# Hurricane Emulator

In [1]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sithom.plot import plot_defaults, label_subplots
from sithom.time import timeit
from src.conversions import si_ify, knots_to_ms
from src.constants import NEW_ORLEANS, DATA_PATH
from src.models.generation import ImpactSymmetricTC, Holland08

In [ ]:
print(prefix, rmax)
rmax_min = 1e3
rmin_min = -1e4

lon_diff = 0.6
point = Point(NEW_ORLEANS.lon + lon_diff, NEW_ORLEANS.lat)
folder = os.path.join(DATA_PATH, "kat_emulation")

if not os.path.exists(folder):
    os.mkdir(folder)
    ImpactSymmetricTC(
        point=point,
        output_direc=os.path.join(folder, prefix + "{:.3f}".format(rmax) + "_kat_rmax"),
        symetric_model=Holland08(rmax=rmax),
    ).run_impact()